In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/566/final/TensorFlow2.0_InceptionV3-master')

In [1]:
# some training parameters
EPOCHS = 50
BATCH_SIZE = 8
NUM_CLASSES = 3
image_height = 299
image_width = 299
channels = 3
save_model_dir = "/content/drive/MyDrive/566/final/TensorFlow2.0_InceptionV3-master/saved_model/model"
dataset_dir = "/content/drive/MyDrive/566/final/divided_dataset/"
train_dir = dataset_dir + "train"
valid_dir = dataset_dir + "valid"
test_dir = dataset_dir + "test"

In [6]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from models import inception_v3
import config
from prepare_data import generate_datasets
import math

def get_model():
    model = inception_v3.InceptionV3(num_class=NUM_CLASSES)

    model.build(input_shape=(None, image_height, image_width, channels))
    model.summary()

    return model


if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)


    # get the original_dataset
    train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets(train_dir,valid_dir,test_dir)


    # create model
    model = get_model()

    # define loss and optimizer
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adadelta()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    valid_loss = tf.keras.metrics.Mean(name='valid_loss')
    valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images, include_aux_logits=True, training=True)
            loss_aux = loss_object(y_true=labels, y_pred=predictions.aux_logits)
            loss = 0.5 * loss_aux + 0.5 * loss_object(y_true=labels, y_pred=predictions.logits)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions.logits)

    @tf.function
    def valid_step(images, labels):
        predictions = model(images, include_aux_logits=False, training=False)
        v_loss = loss_object(labels, predictions)

        valid_loss(v_loss)
        valid_accuracy(labels, predictions)

    # start training
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        valid_loss.reset_states()
        valid_accuracy.reset_states()
        step = 0
        for images, labels in train_dataset:
            step += 1
            train_step(images, labels)
            print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                     config.EPOCHS,
                                                                                     step,
                                                                                     math.ceil(train_count / config.BATCH_SIZE),
                                                                                     train_loss.result(),
                                                                                     train_accuracy.result()))

        for valid_images, valid_labels in valid_dataset:
            valid_step(valid_images, valid_labels)

        print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
              "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                  config.EPOCHS,
                                                                  train_loss.result(),
                                                                  train_accuracy.result(),
                                                                  valid_loss.result(),
                                                                  valid_accuracy.result()))

    model.save_weights(filepath=save_model_dir, save_format='tf')

Model: "inception_v3_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocess_1 (Preprocess)   multiple                  173872    
                                                                 
 sequential_3 (Sequential)   (None, 38, 38, 288)       822896    
                                                                 
 sequential_4 (Sequential)   (None, 18, 18, 768)       7997312   
                                                                 
 inception_aux_1 (InceptionA  multiple                 2562691   
 ux)                                                             
                                                                 
 sequential_5 (Sequential)   (None, 8, 8, 2048)        11065984  
                                                                 
 average_pooling2d_13 (Avera  multiple                 0         
 gePooling2D)                                       

In [7]:
loss_object = tf.keras.metrics.SparseCategoricalCrossentropy()
test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def test_step(images, labels):
  predictions = model(images, include_aux_logits=False, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

for test_images, test_labels in test_dataset:
  test_step(test_images, test_labels)
  print("loss: {:.5f}, test accuracy: {:.5f}".format(test_loss.result(),
                                                           test_accuracy.result()))

print("The accuracy on test set is: {:.3f}%".format(test_accuracy.result()*100))

loss: 11.96437, test accuracy: 0.00000
loss: 11.84746, test accuracy: 0.00000
loss: 11.60177, test accuracy: 0.00000
loss: 10.92788, test accuracy: 0.18750
loss: 10.17590, test accuracy: 0.35000
loss: 9.48048, test accuracy: 0.45833
loss: 8.86546, test accuracy: 0.53571
loss: 8.46888, test accuracy: 0.51562
loss: 8.09548, test accuracy: 0.55556
loss: 7.78769, test accuracy: 0.55000
loss: 7.52819, test accuracy: 0.56818
loss: 7.30400, test accuracy: 0.57292
loss: 7.10695, test accuracy: 0.58654
loss: 6.93403, test accuracy: 0.58929
loss: 6.80468, test accuracy: 0.57500
loss: 6.68670, test accuracy: 0.57812
loss: 6.58557, test accuracy: 0.58088
loss: 6.49201, test accuracy: 0.59028
loss: 6.40486, test accuracy: 0.59868
loss: 6.32359, test accuracy: 0.60625
loss: 6.23943, test accuracy: 0.61905
loss: 6.15734, test accuracy: 0.63068
loss: 6.07773, test accuracy: 0.64130
loss: 6.00399, test accuracy: 0.64583
loss: 5.93234, test accuracy: 0.65000
loss: 5.86296, test accuracy: 0.65385
loss: 5